### Imports

In [1]:
from tqdm import tqdm
import random
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import gymnasium as gym
import gym_mtsim
sys.path.append("C:/Users/WilliamFetzner/Documents/Trading/")
from gym_mtsim_forked.gym_mtsim.data import FOREX_DATA_PATH_TRAIN, FOREX_DATA_PATH_TEST, FOREX_DATA_PATH, STOCKS_DATA_PATH, FOREX_DATA_PATH_PRODUCTION, FOREX_DATA_PATH_1HR
from gym_mtsim import OrderType, Timeframe, MtEnv, MtSimulator
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, STATUS_FAIL
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
from datetime import datetime, timedelta
import time
import torch
import pickle
import fx_rl

c:\Users\WilliamFetzner\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment forex-hedge-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Users\WilliamFetzner\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment forex-unhedge-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Users\WilliamFetzner\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment stocks-hedge-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Users\WilliamFetzner\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment stocks-unhedge-v0 alr

# Parameters

In [ ]:
with open(FOREX_DATA_PATH, 'rb') as f:
    symbols_1hr = pickle.load(f)
# convert symbols_1hr to a pd.dataframe
symbols_1hr[1]['EURUSD'].index = pd.to_datetime(symbols_1hr[1]['EURUSD'].index)
max_date_1hr = symbols_1hr[1]['EURUSD'].index.max()

In [ ]:
slices_1hr = fx_rl.slices_finder(symbols_1hr[1]['EURUSD'], max_date_1hr, testing_needed=False)

### Create Env

In [ ]:
# class MyMtEnv(gym_mtsim.MtEnv):
#     # _get_modified_volume = fx_rl.my_get_modified_volume
#     _get_prices = fx_rl.my_get_prices

In [ ]:
# sim_train = gym_mtsim.MtSimulator(
#     unit='USD',
#     balance=200000.,
#     leverage=100.,
#     stop_out_level=0.2,
#     hedge=True,
#     symbols_filename=FOREX_DATA_PATH
# )

In [ ]:
# env_train = gym_mtsim.MtEnv(
#     original_simulator=sim_train,
#     trading_symbols=['EURUSD'],
#     window_size = 10,
#     time_points=slices_1hr[0],
#     hold_threshold=0.5,
#     close_threshold=0.5,
#     fee=lambda symbol: {
#         # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
#         'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
#         # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
#     }[symbol],
#     symbol_max_orders=2,
#     multiprocessing_processes=2
# )

In [ ]:
# sim_validation = gym_mtsim.MtSimulator(
#     unit='USD',
#     balance=200000.,
#     leverage=100.,
#     stop_out_level=0.2,
#     hedge=True,
#     symbols_filename=FOREX_DATA_PATH
# )

# env_validation = gym_mtsim.MtEnv(
#     original_simulator=sim_validation,
#     trading_symbols=['EURUSD'],
#     window_size = 10,
#     time_points=slices_1hr[1],
#     hold_threshold=0.5,
#     close_threshold=0.5,
#     fee=lambda symbol: {
#         # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
#         'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
#         # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
#     }[symbol],
#     symbol_max_orders=2,
#     multiprocessing_processes=2
# )

In [ ]:
# sim_validation_2 = gym_mtsim.MtSimulator(
#     unit='USD',
#     balance=200000.,
#     leverage=100.,
#     stop_out_level=0.2,
#     hedge=True,
#     symbols_filename=FOREX_DATA_PATH
# )

# env_validation_2 = gym_mtsim.MtEnv(
#     original_simulator=sim_validation_2,
#     trading_symbols=['EURUSD'],
#     window_size = 10,
#     time_points=slices_1hr[1],
#     hold_threshold=0.5,
#     close_threshold=0.5,
#     fee=lambda symbol: {
#         # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
#         'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
#         # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
#     }[symbol],
#     symbol_max_orders=2,
#     multiprocessing_processes=2
# )

### Define Functions

In [ ]:
def print_stats(reward_over_episodes, printing_name):
    """  Print Reward  """

    avg_rewards = np.mean(reward_over_episodes)
    min_rewards = np.min(reward_over_episodes)
    max_rewards = np.max(reward_over_episodes)

    print (f'Min. {printing_name}          : {min_rewards:>10.3f}')
    print (f'Avg. {printing_name}          : {avg_rewards:>10.3f}')
    print (f'Max. {printing_name}          : {max_rewards:>10.3f}')

    return min_rewards, avg_rewards, max_rewards

def my_profit_calculation(env_orders, stop_loss):
        # env_orders = env_testing.render()['orders']
        # stop_loss = 0.001
        mean_value = env_orders['Volume'].mean()

        # # Normalize the column to have a mean of 1
        env_orders.loc[:, 'Volume'] = round((env_orders['Volume'] / mean_value), 2)
        # add a column for when the difference between the Entry Price and the Exit Price is greater than stop_loss
        env_orders.loc[:, 'stoploss_hit'] = np.where((env_orders['Type'].str.strip() == 'Buy') &
                                                        ((env_orders['Entry Price'] - env_orders['Exit Price']) > stop_loss),
                                                        1, np.where((env_orders['Type'].str.strip() == 'Sell') &
                                                                        ((env_orders['Exit Price'] - env_orders['Entry Price']) > stop_loss),
                                                                        1, 0))
        env_orders.loc[:, 'Exit Price'] = np.where((env_orders['Type'].str.strip() == 'Buy') & (env_orders['stoploss_hit'] == 1),
                                                        env_orders['Entry Price'] - stop_loss,
                                                        np.where((env_orders['Type'].str.strip() == 'Sell') & (env_orders['stoploss_hit'] == 1),
                                                                env_orders['Entry Price'] + stop_loss, env_orders['Exit Price']))
        env_orders.loc[:, 'Profit'] = np.where((env_orders['Type'].str.strip() == 'Buy'),
                                                        ((env_orders['Exit Price'] - (env_orders['Fee']/2)) - 
                                                        (env_orders['Entry Price'] + (env_orders['Fee']/2)))
                                                                * 100_000 * env_orders['Volume'], 
                                                        np.where((env_orders['Type'].str.strip() == 'Sell'),
                                                                ((env_orders['Entry Price'] - (env_orders['Fee']/2)) - 
                                                                (env_orders['Exit Price'] + (env_orders['Fee']/2)))
                                                                * 100_000 * env_orders['Volume'], np.nan))
        total_reward = env_orders.loc[:, 'Profit'].sum()
        # Calculate Gross Profit
        gross_profit = env_orders.loc[env_orders['Profit'] > 0, 'Profit'].sum()

        # Calculate Gross Loss
        gross_loss = env_orders.loc[env_orders['Profit'] < 0, 'Profit'].abs().sum()

        # Calculate Profit Factor
        profit_factor = gross_profit / gross_loss if gross_loss != 0 else 0

        profit_factor = profit_factor - 1

        return profit_factor, total_reward

# ProgressBarCallback for model.learn()
class ProgressBarCallback(BaseCallback):

    def __init__(self, check_freq: int, verbose: int = 1):
        super().__init__(verbose)
        self.check_freq = check_freq

    def _on_training_start(self) -> None:
        """
        This method is called before the first rollout starts.
        """
        self.progress_bar = tqdm(total=self.model._total_timesteps, desc="model.learn()")

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            self.progress_bar.update(self.check_freq)
        return True
    
    def _on_training_end(self) -> None:
        """
        This event is triggered before exiting the `learn()` method.
        """
        self.progress_bar.close()


In [ ]:
# TRAINING + TEST
def train_val_model(model, model_policy, env_tr, env_val, seed, steps_str, lr, gamma_param, entropy, total_learning_timesteps=10_000):
    """
    Trains and validates a model using the Proximal Policy Optimization (PPO) algorithm.

    Args:
        model (object): The model to be trained.
        model_policy (object): The policy used by the model.
        env_tr (object): The training environment.
        env_val (object): The validation environment.
        seed (int): The random seed for reproducibility.
        steps_str (str): A string representing the number of steps.
        window_size_param (int): The window size parameter.
        lr (float): The learning rate.
        gamma_param (float): The gamma parameter.
        entropy (float): The entropy coefficient.
        total_learning_timesteps (int, optional): The total number of learning timesteps. Defaults to 10,000.

    Returns:
        tuple: A tuple containing the reward over validations, orders over validations, and the model dictionary.
    """
    # reproduce training and test
    print('-' * 80)
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    #model_dict = {}
    # env_tr.window_size = window_size_param
    print(f'entropy: {entropy}, learning rate: {lr}, gamma: {gamma_param}')
    # eval_callback = EvalCallback(env_tr, log_path='./logs/', eval_freq=1000)
    obs_tr, info_tr = env_tr.reset(seed=seed)
    model = PPO(model_policy, env_tr, verbose=0, ent_coef=entropy, learning_rate=lr)#, gamma=gamma_param, 
    
    # custom callback for 'progress_bar'
    model.learn(total_timesteps=total_learning_timesteps)#, callback=ProgressBarCallback(100))

    reward_over_validations = []
    orders_over_validations = []
    profit_over_validations = []

    for episode in range(0, 10):
        obs_val, info_val = env_val.reset(seed=seed)

        total_reward = 0
        done_val = False

        while not done_val:
            action, _states = model.predict(obs_val)
            obs_val, reward_val, terminated_val, truncated_val, info_val = env_val.step(action)
            done_val = terminated_val or truncated_val

            total_reward += reward_val
            if done_val:
                break
        try:
            orders_made_in_episode = env_val.render()['orders']
            order_len = len(orders_made_in_episode)
            total_reward, total_profit = my_profit_calculation(orders_made_in_episode, 0.001)
            
        except Exception as e:
            print('There were not any orders produced by the model')
            print(f'Here is the error {e}')
            order_len = 0
            total_profit = 0

        # model_dict[f'model_{episode}'] = model
        # model.save(f'best_hyperparameters/models_4_26_24/model_{episode}.pkl')

        reward_over_validations.append(total_reward) 
        profit_over_validations.append(total_profit)   
        orders_over_validations.append(order_len)  


        # if episode % 1 == 0:
        avg_reward = np.mean(reward_over_validations)
        avg_orders = np.mean(orders_over_validations)
        avg_profit = np.mean(profit_over_validations)
        print(f'Episode: {episode}, Avg. Reward: {avg_reward:.3f}, # of orders: {avg_orders:.3f}, avg Profit: {avg_profit:.3f}')
    model.save(f'models_{max_date_1hr.date()}/model_{steps_str}_{max_date_1hr.date()}.pkl')
    return reward_over_validations, orders_over_validations, profit_over_validations#, model_dict

## Objective Function

In [ ]:
# seed = 2024  # random seed
# total_num_episodes = 10

# # print ("env_name                 :", env_name)
# print ("seed                     :", seed)

# # INIT matplotlib
# plot_settings = {}
# plot_data = {'x': [i for i in range(1, total_num_episodes + 1)]}

# # learning_timesteps_list_in_K = [25]#, 50, 100]
# # learning_timesteps_list_in_K = [50, 250, 500]
# # learning_timesteps_list_in_K = [500, 1000, 3000, 5000]

# # RL Algorithms: https://stable-baselines3.readthedocs.io/en/master/guide/algos.html

# timesteps_models_dict = {}
# def objective(params):
#     learning_timesteps = 100 #params['learning_timesteps']
#     ent_coef = params['ent_coef']
#     gamma = params['gamma'] #0.99 #
#     learning_rate = params['learning_rate']#0.0003#

#     if learning_rate > 0.08:
#         print(f'Learning rate too high: {learning_rate}')
#         return {'loss': None, 'status': STATUS_FAIL, 'eval_time': time.time(), 'parameters': params}
#     if ent_coef > 0.1:
#         print(f'Entropy too high: {ent_coef}')
#         return {'loss': None, 'status': STATUS_FAIL, 'eval_time': time.time(), 'parameters': params}

#     total_learning_timesteps = learning_timesteps * 1000
#     step_key = f'{learning_timesteps}K'
#     policy_dict = PPO.policy_aliases
#     policy = policy_dict.get('MultiInputPolicy')
#     class_name = type(PPO).__qualname__
#     plot_key = f'{class_name}_rewards_'+step_key
#     try:
#         rewards, orders, profits = train_val_model(PPO, policy, env_train, env_validation, seed, step_key,  
#                                                     learning_rate, gamma, ent_coef, total_learning_timesteps)
#     except Exception as e:
#         print(f'''there was an error {e} with those parameters: timesteps: {learning_timesteps}, \n
#               ent_coef: {ent_coef}, gamma: {gamma}, learning_rate: {learning_rate}''')
#         return {'loss': None, 'status': STATUS_FAIL, 'eval_time': time.time(), 'parameters': params}
#     # timesteps_models_dict[step_key] = models_dict
#     min_rewards, avg_rewards, max_rewards, = print_stats(rewards, 'Reward')
#     print_stats(orders, 'Orders')
#     label = f'Avg. {avg_rewards:>7.2f} : {class_name} - {step_key}'
#     plot_data[plot_key] = rewards
#     plot_settings[plot_key] = {'label': label}
#     params['avg_orders'] = np.mean(orders)
#     params['profits'] = np.mean(profits)       

#     return {'loss': -avg_rewards, 'status': STATUS_OK, 'eval_time': time.time(), 'parameters': params} 

seed                     : 2024


In [ ]:
# best_hyperparameters_current_week = pd.read_excel('best_hyperparameter_search_results.xlsx')

In [ ]:
trials_latest_week = pickle.load(open(f'gym_mtsim_forked/examples/hyperopt/trials_5_17_search_next_week.pkl', 'rb'))
trials_latest_week_all_results = trials_latest_week.results
print(len(trials_latest_week_all_results),
trials_latest_week_all_results[0])

250 {'loss': -28.989188416963536, 'status': 'ok', 'eval_time': 1715980468.2894373, 'parameters': {'ent_coef': 0.0385069270635906, 'gamma': 0.9568912588294178, 'learning_rate': 0.01866046145280669, 'avg_orders': 36.3, 'rewards': 0.05394586565518563}}


In [ ]:
# trials_latest_week.best_trial

{'state': 2,
 'tid': 224,
 'spec': None,
 'result': {'loss': -1177.8298907055178,
  'status': 'ok',
  'eval_time': 1716039083.017912,
  'parameters': {'ent_coef': 0.0011367890106411677,
   'gamma': 0.9416691383716818,
   'learning_rate': 0.026460089849855668,
   'avg_orders': 24.8,
   'rewards': 1.9290342670391543}},
 'misc': {'tid': 224,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'ent_coef': [224], 'gamma': [224], 'learning_rate': [224]},
  'vals': {'ent_coef': [0.0011367890106411677],
   'gamma': [0.9416691383716818],
   'learning_rate': [0.026460089849855668]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2024, 5, 18, 13, 28, 9, 558000),
 'refresh_time': datetime.datetime(2024, 5, 18, 13, 31, 23, 17000)}

In [ ]:
best_hyperparameters_current_week = pd.DataFrame()
new_dict = {}
for idx, result in enumerate(trials_latest_week_all_results):
    new_dict['loss'] = result['loss']
    new_dict['status'] = result['status']
    new_dict['learning_rate'] = result['parameters']['learning_rate']
    new_dict['ent_coef'] = result['parameters']['ent_coef']
    new_dict['gamma'] = result['parameters']['gamma']
    try:
        new_dict['orders'] = result['parameters']['avg_orders']
        new_dict['rewards'] = result['parameters']['rewards']
    except Exception as e: 
        new_dict['orders'] = 0
        new_dict['rewards'] = 0
    new_row = pd.DataFrame(new_dict, index=[idx])
    best_hyperparameters_current_week = pd.concat([best_hyperparameters_current_week, new_row], axis=0)
best_hyperparameters_current_week


,loss,status,learning_rate,ent_coef,gamma,orders,rewards
0,-28.989188,ok,0.018660,0.038507,0.956891,36.3,0.053946
1,10.609816,ok,0.003202,0.009890,0.939635,0.1,-0.100000
2,-13.247216,ok,0.025022,0.038900,0.937440,39.1,0.016948
3,29.697417,ok,0.002975,0.007454,0.960527,104.0,-0.006706
4,19.699558,ok,0.002955,0.018230,0.938070,0.8,0.553679
...,...,...,...,...,...,...,...
245,-283.258543,ok,0.037056,0.000070,0.931006,20.9,0.427490
246,51.063109,ok,0.022112,0.014385,0.942434,37.0,-0.028249
247,133.799695,ok,0.015936,0.021098,0.934058,45.0,-0.069388
248,98.009133,ok,0.038058,0.001041,0.939902,38.2,-0.087719


In [ ]:
best_hyperparameters_current_week_success = best_hyperparameters_current_week[best_hyperparameters_current_week['status'] == 'ok']
best_hyperparameters_current_week_success_negative = best_hyperparameters_current_week_success[best_hyperparameters_current_week_success['loss'] < 0]
best_hyperparameters_current_week_success_negative = best_hyperparameters_current_week_success_negative.sort_values(by='loss', ascending=True)
best_hyperparameters_current_week_success_negative

,loss,status,learning_rate,ent_coef,gamma,orders,rewards
224,-1177.829891,ok,0.026460,0.001137,0.941669,24.8,1.929034
114,-861.134723,ok,0.032388,0.000059,0.935070,2.0,0.033623
89,-678.911910,ok,0.031810,0.004452,0.943984,48.2,0.537641
180,-653.672668,ok,0.018963,0.000079,0.932250,108.2,0.281889
60,-545.591384,ok,0.027795,0.002588,0.935848,26.3,0.596425
...,...,...,...,...,...,...,...
197,-5.636681,ok,0.024878,0.027399,0.928146,38.9,0.007843
113,-5.636681,ok,0.024363,0.037461,0.949387,38.9,0.007843
229,-5.636681,ok,0.022883,0.048445,0.952559,38.9,0.007843
75,-5.636681,ok,0.032488,0.000956,0.946843,38.9,0.007843


In [ ]:
len(best_hyperparameters_current_week_success_negative)

66

In [ ]:
# best_hyperparameters_current_week_success_negative.loc[116,'ent_coef']

0.0109201230283189

In [ ]:
# profits_dict = {}
# rewards_dict = {}
# for idx in tqdm(best_hyperparameters_current_week_success_negative.index):
#     parameters = {
#         # 'window_size': 10,
#         # 'learning_timesteps': 25,
#         'ent_coef': best_hyperparameters_current_week_success_negative.loc[idx,'ent_coef'],
#         'gamma': best_hyperparameters_current_week_success_negative.loc[idx,'gamma'],
#         'learning_rate': best_hyperparameters_current_week_success_negative.loc[idx,'learning_rate']
#     }
#     rewards_250, orders_250, profits = train_val_model(PPO, 'MultiInputPolicy', env_train, env_validation, seed, str(idx),  
#                                                 parameters['learning_rate'], parameters['gamma'], parameters['ent_coef'], 250_000)
#     print_stats(profits, 'Profits')
#     print_stats(rewards_250, 'Rewards')
#     profits_dict[f'{idx}'] = profits
#     rewards_dict[f'{idx}'] = rewards_250

# # I want to know out of all the ones that I tested, are there any that don't go negative in profits, and use that model, then use any of those models in the 1000 run test below

    

In [ ]:
total_learning_timesteps = 250 * 1_000

In [ ]:
def env_fx(slices):
    sim_training = gym_mtsim.MtSimulator(
        unit='USD',
        balance=200000.,
        leverage=100.,
        stop_out_level=0.2,
        hedge=True,
        symbols_filename=FOREX_DATA_PATH
    )
    env_training = gym_mtsim.MtEnv(
        original_simulator=sim_training,
        trading_symbols=['EURUSD'],
        window_size = 10,
        time_points= slices[0],
        hold_threshold=0.5,
        close_threshold=0.5,
        fee=lambda symbol: {
            # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
            'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
            # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
        }[symbol],
        symbol_max_orders=2,
        multiprocessing_processes=2
    )
    sim_validation = gym_mtsim.MtSimulator(
        unit='USD',
        balance=200000.,
        leverage=100.,
        stop_out_level=0.2,
        hedge=True,
        symbols_filename=FOREX_DATA_PATH
    )
    env_validation = gym_mtsim.MtEnv(
        original_simulator=sim_validation,
        trading_symbols=['EURUSD'],
        window_size = 10,
        time_points= slices[1],
        hold_threshold=0.5,
        close_threshold=0.5,
        fee=lambda symbol: {
            # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
            'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
            # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
        }[symbol],
        symbol_max_orders=2,
        multiprocessing_processes=2
    )
    return env_training, env_validation

In [9]:
# find all of the models that are in the /models_2024_05_17 folder
import os

folder_path = 'C:/Users/WilliamFetzner/Documents/Trading/models_2024-05-17'

# Get the list of files in the folder
file_list = os.listdir(folder_path)
file_list[0]

'model_0_2024-05-17.pkl'

In [12]:
file_list

['model_0_2024-05-17.pkl',
 'model_114_2024-05-17.pkl',
 'model_11_2024-05-17.pkl',
 'model_134_2024-05-17.pkl',
 'model_137_2024-05-17.pkl',
 'model_140_2024-05-17.pkl',
 'model_147_2024-05-17.pkl',
 'model_150_2024-05-17.pkl',
 'model_15_2024-05-17.pkl',
 'model_162_2024-05-17.pkl',
 'model_165_2024-05-17.pkl',
 'model_168_2024-05-17.pkl',
 'model_173_2024-05-17.pkl',
 'model_17_2024-05-17.pkl',
 'model_180_2024-05-17.pkl',
 'model_182_2024-05-17.pkl',
 'model_185_2024-05-17.pkl',
 'model_189_2024-05-17.pkl',
 'model_193_2024-05-17.pkl',
 'model_194_2024-05-17.pkl',
 'model_198_2024-05-17.pkl',
 'model_20_2024-05-17.pkl',
 'model_215_2024-05-17.pkl',
 'model_224_2024-05-17.pkl',
 'model_226_2024-05-17.pkl',
 'model_227_2024-05-17.pkl',
 'model_236_2024-05-17.pkl',
 'model_244_2024-05-17.pkl',
 'model_245_2024-05-17.pkl',
 'model_37_2024-05-17.pkl',
 'model_41_2024-05-17.pkl',
 'model_49_2024-05-17.pkl',
 'model_50_2024-05-17.pkl',
 'model_53_2024-05-17.pkl',
 'model_54_2024-05-17.pkl

In [ ]:
# get the model number from the file name when the file name has the format 'model_##_2024-05-17.pkl' where the ## is the model number
model_nums = [x.split('_')[1] for x in file_list]

In [ ]:
# test which version of the model to take, does it make a difference whether I use model_0 or model_9
seed=2024
num_episodes = 1_000

total_reward = 0
done_test = False
reward_over_tests = {}
time_now = datetime.now()
# time_now = datetime.now()
# slices_ready = [0, 0]
# slices_ready[1] = slices_1hr[1][mins:maxes]
# timestep_max = slices_ready[1][0]
# slices_1hr_ready = slices_1hr[1][slices_1hr[1] < timestep_max]
# slices_ready[0] = slices_1hr[0].append(slices_1hr_ready)

train_env, env_validation_2 = env_fx(slices_1hr)
obs_test, info_test = env_validation_2.reset(seed=seed)
obs_tr, info_tr = train_env.reset(seed=seed)

for num, model in tqdm(zip(model_nums, file_list)):

    # model_ppo = PPO('MultiInputPolicy', train_env, verbose=0, ent_coef=m_ppo_ent_coef, learning_rate=m_ppo_lr, gamma=m_ppo_gamma)
    obs_tr, info_tr = train_env.reset(seed=seed)
    model_ppo = PPO.load(f'models_2024-05-17/{model}', train_env)
    # model_ppo.learn(total_timesteps=total_learning_timesteps, callback=ProgressBarCallback(100))   

    rewards = []
    for episode in range(num_episodes):
        obs_test, info_test = env_validation_2.reset(seed=seed)
        # model_ppo = PPO.load(f'gym_mtsim_forked/gym_mtsim/data/model_0.pkl', train_env)
        done_test = False
        while not done_test:
            action, _states = model_ppo.predict(obs_test)
            obs_test, reward_test, terminated_test, truncated_test, info_test = env_validation_2.step(action)
            done_test = terminated_test or truncated_test
            
            total_reward += reward_test
            if done_test:
                break
        try:
            orders_made_in_episode_test = env_validation_2.render()['orders']
            # orders_over_validations_dfs[f'{episode}'] = orders_made_in_episode_test
            order_len = len(orders_made_in_episode_test)
            total_reward, total_profit = my_profit_calculation(orders_made_in_episode_test, 0.001)
            rewards.append(total_profit)
        except Exception as e:
            print(f'There were not any orders produced by the model. Error = {e}')
            order_len = 0
    reward_over_tests[f'model_{num}'] = rewards
# print(f'Finished day_{mins} in {int((datetime.now() - time_now).seconds/60)} minutes')


model.learn(): 251900it [15:09, 276.93it/s]                            


Finished day_0 in 16 minutes


model.learn(): 251900it [19:11, 218.80it/s]                            


Finished day_24 in 20 minutes


model.learn(): 251900it [16:19, 257.06it/s]                            


Finished day_48 in 17 minutes


model.learn(): 251900it [19:43, 212.78it/s]                            


Finished day_72 in 21 minutes


model.learn(): 251900it [10:48, 388.61it/s]                            


Finished day_96 in 12 minutes


In [90]:
reward_over_tests

{'day_0': [10.6813515640347,
  48.04330829807404,
  -191.53744874076398,
  -118.44622585635011,
  232.40551611038995,
  -148.8108642554069,
  -15.945093577328649,
  114.53301237723701,
  -325.2414328418425,
  0.34967842044665076,
  -232.72037284358672,
  -8.06917395315132,
  -41.52807969368678,
  -190.1335603497145,
  6.937602213685786,
  -178.2474115976873,
  63.825145971121664,
  -220.87297023110077,
  22.80322142484968,
  -212.78198770273704,
  81.82218393363152,
  35.84928243463013,
  147.1215374282573,
  -113.6929575785488,
  99.47236995668639,
  231.84057342200413,
  -203.9845607345381,
  68.18505319424428,
  -353.8853164768874,
  151.93459829985213,
  55.053934973381395,
  -243.30386907380762,
  -75.9459386850514,
  -171.53137416416888,
  -141.28821145511841,
  -259.0315439435802,
  -68.15751121794776,
  -242.50977208328806,
  -198.80733554329396,
  -198.78585402690604,
  49.6408342506527,
  -221.52679873626013,
  -76.42132173908412,
  -6.536517389665649,
  13.254622340741491,
 